# MNIST data classification: Keras and input pipeline

This notebook shows how to use input pipelines with Keras.

In [3]:
import numpy as np
import math
import tensorflow as tf

In [4]:
tf.version.VERSION

'2.3.0'

In [5]:
def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    features = tf.io.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.int64),
        })
    label = tf.cast(features['label'], tf.int32)
    image = tf.io.decode_raw(features['image_raw'], tf.uint8)
    return image, label


def normalize(image, label):
    """Convert `image` from [0, 255] -> [-0.5, 0.5] floats."""
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label

In [6]:
dataset = tf.data.TFRecordDataset('../tfrecords/train.tfrecords')
dataset = dataset.map(decode)
dataset = dataset.map(normalize)
dataset = dataset.shuffle(6000)
dataset = dataset.batch(256)

In [7]:
test_set = tf.data.TFRecordDataset('../tfrecords/test.tfrecords')
test_set = test_set.map(decode)
test_set = test_set.map(normalize)
test_set = test_set.batch(256)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [9]:
opt = tf.keras.optimizers.Adam(0.001)

In [10]:
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
fit = model.fit(dataset,
                epochs=3,
                validation_data=test_set)

Epoch 1/3
235/235 [==============================] - 4s 15ms/step - loss: 0.5150 - accuracy: 0.8537 - val_loss: 0.3079 - val_accuracy: 0.9116
Epoch 2/3
235/235 [==============================] - 3s 13ms/step - loss: 0.2881 - accuracy: 0.9172 - val_loss: 0.2486 - val_accuracy: 0.9287
Epoch 3/3
235/235 [==============================] - 3s 14ms/step - loss: 0.2363 - accuracy: 0.9316 - val_loss: 0.2093 - val_accuracy: 0.9382
